# Challenge 1: Determining the Most Important Features in Diamond Pricing
The main aim of this notebook is understanding the most important factors influencing diamond prices from a dataset of 5000 diamonds priced by an expert.
Throughout the notebook we will:
- explore the features available for each diamond
- pre-process the dataset
- visually explore the dataset
- explore the most important features in diamond pricing

## Loading the Dataset


In [135]:
# clone the dataset from github
!git clone https://github.com/dpaletti/xtream-assignment/

Cloning into 'xtream-assignment'...
remote: Enumerating objects: 296, done.
remote: Counting objects: 100% (246/246), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 296 (delta 114), reused 227 (delta 99), pack-reused 50
Receiving objects: 100% (296/296), 4.32 MiB | 3.83 MiB/s, done.
Resolving deltas: 100% (121/121), done.


In [120]:
# load the dataset from the csv file
import pandas as pd
dataset = pd.read_csv("./xtream-assignment/datasets/diamonds/diamonds.csv")

## Understanding the Dataset


### High-Level View
- **No missing data**: all the columns have 5000 datapoints
- **Price**:  the minimum price is -1 which raises an alarm bell
- **Sizes**: some diamonds have x, y or z equals to 0 which is equally problematic

In [121]:
dataset.describe()

,carat,depth,table,price,x,y,z
count,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000
mean,0.794486,61.711660,57.447060,3925.53940,5.725188,5.727744,3.533076
std,0.468424,1.446206,2.258999,3975.45212,1.119156,1.112106,0.690334
min,0.230000,44.000000,51.600000,-1.00000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,936.00000,4.700000,4.710000,2.900000
50%,0.700000,61.800000,57.000000,2392.50000,5.690000,5.700000,3.530000
75%,1.040000,62.500000,59.000000,5369.25000,6.540000,6.540000,4.030000
max,4.130000,70.200000,95.000000,18787.00000,10.000000,9.850000,6.430000


### Diamond Features

We enumerate all the features available for a generic diamond.

Features:
- **carat**: diamond weight measured as multiples of 200mg
- **cut**: grading system that measures how close a cut stone comes to an ideal set of proportions
- **color**: grading system that measures how close a diamond is to being colorless
- **clarity**: grading system that measures size, position and number of inclusions or blemishes
- **depth**: diamond depth in mm
- **table**: diamond table in mm
- **x, y, z**: diamond's phisical dimensions in mm

Each diamond has been given a **price**.

In [122]:
target = "price"
features = list(filter(lambda x: x != target, dataset.columns))
categorical_features = ["cut", "color", "clarity"]

### Cut Grades
All features are well explained except for cut grades.

[The first hit on google](https://www.gemsociety.org/article/gias-new-diamond-cut-grading-system/) reports grades going from *Excellent* to *Poor* but a rapid glimpse at the dataset unveils different grades such as *Ideal*. We will now look for all the cut grades available in the dataset.

**Cut grades**:
- Ideal
- Premium 
- Very Good 
- Good
- Fair

**Assumption**: no cut grade is missing in the dataset provided

In [123]:
cut_grades = list(dataset["cut"].unique())

## Dataset Preprocessing


### Cleaning
**Remove incorrect diamonds**:
- remove all diamonds with price less than or equal to 0
- remove all diamonds with some 0-length dimension


In [124]:
cleaned_dataset = dataset.drop(dataset[dataset.price <=0 ].index)
cleaned_dataset = cleaned_dataset.drop(cleaned_dataset[(cleaned_dataset["x"] == 0) | (cleaned_dataset["y"] == 0) | (cleaned_dataset["z"] == 0)].index)
cleaned_dataset.describe()

,carat,depth,table,price,x,y,z
count,4989.000000,4989.000000,4989.000000,4989.000000,4989.000000,4989.000000,4989.000000
mean,0.794117,61.710844,57.446242,3930.581880,5.726232,5.728821,3.533678
std,0.467981,1.445563,2.259752,3970.923311,1.116257,1.109167,0.688437
min,0.230000,44.000000,51.600000,351.000000,3.860000,3.840000,1.410000
25%,0.400000,61.000000,56.000000,942.000000,4.700000,4.710000,2.900000
50%,0.700000,61.800000,57.000000,2398.000000,5.690000,5.700000,3.530000
75%,1.040000,62.500000,59.000000,5376.000000,6.540000,6.540000,4.030000
max,4.130000,70.200000,95.000000,18787.000000,10.000000,9.850000,6.430000


### Converting Scales to Consecutive Integers
Cut, color and clarity are defined as scales. 

Machine learning models need integer features to get a sense of the meaning of these scales. Thus, we convert all the scales to consecutive integers where **higher integers represent higher positions in the scale**.

Take clarity grades as an example:
- IF -> 7
- VVS1 -> 6
- VVS2 -> 5
- VS1 -> 4
- VS2 -> 3
- SI1 -> 2
- SI2 -> 1
- I1 -> 0

In [125]:
import string

cut_mapping = {
    "Ideal": 4,
    "Premium": 3,
    "Very Good": 2,
    "Good": 1,
    "Fair": 0,
}

color_mapping = {color_grade: value 
                 for value, color_grade 
                 in enumerate(string.ascii_uppercase[3:])}

clarity_mapping = {
    "IF": 7,
    "VVS1": 6,
    "VVS2": 5,
    "VS1": 4,
    "VS2": 3,
    "SI1": 2,
    "SI2": 1,
    "I1": 0
}

mappings = [cut_mapping, color_mapping, clarity_mapping]

In [126]:
from functools import reduce
from typing import Dict, List

def convert_to_ordinal(dataset: pd.DataFrame, feature_map: Dict[str, int]) -> pd.DataFrame:
  """Convert a categorical feature to ordinal in a given dataset"""
  return dataset.replace(feature_map)

def convert_all_to_ordinal(dataset: pd.DataFrame, 
                           feature_maps: List[Dict[str, int]]) -> pd.DataFrame:
  """Convert a list of categorical features to ordinal in a given dataset"""
  return reduce(convert_to_ordinal, feature_maps, dataset)


In [127]:
dataset_no_categorical = convert_all_to_ordinal(cleaned_dataset, mappings)

### Normalization: Pushing All Values to the (0,1) Range
Some machine learning models may be negatively influenced by the difference in magnitude among the features. 

They may **privilege some feature only for their magnitude** and not for the role they play in determining the price. Therefore, we push all features to the (0,1) range to avoid such issues.


In [128]:
normalized_dataset=(dataset_no_categorical-dataset_no_categorical.min())/(dataset_no_categorical.max()-dataset_no_categorical.min())

### Split Dataset
Dataset is now split in **training** set and **test** set. The training set is the data that the machine learning model "learns" from. The test set is used for evaluating model performance on unseen data. 

In [129]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(normalized_dataset, test_size=0.2, random_state=0)
samples_train = train.drop([target], axis=1)
targets_train = train[target]
samples_test = test.drop([target], axis=1)
targets_test = test[target]


## Visualizing the Dataset
Visualization allows us to **get an idea of the diamonds in the dataset** and thus a preliminary intuition of what we can expect from our machine learning models.


In [130]:
from plotly.graph_objs._figure import Figure
from typing import Optional
import plotly.express as px 


def make_barchart(dataset: pd.DataFrame, feature: str, color: str) -> Figure:
  """Plot a feature from a given dataset as a barchart"""
  barchart = px.bar(dataset, x=feature)
  barchart.update_traces(marker_color=color)
  return barchart

def make_histogram(dataset: pd.DataFrame,
                   feature: str,
                   color: str) -> Figure:
  """Plot a feature from a given dataset as a histogram"""
  histogram = px.histogram(dataset, x=feature)
  histogram.update_traces(marker_color=color)
  return histogram

### Prices
The histogram below represents the **price distribution** among the diamonds in the dataset. 

On the **y-axis we represent the number of diamonds in the same price range** while on the **x-axis we have the actual price**. For example, by hovering the mouse on the highest bar we can see it represents the price range between 500\$ and 999\$ which covers the price of 1198/5000 diamonds in the dataset.

From this plot we get the clear idea that the **dataset is not uniform**, which means that the price is not uniformly distributed with respect to samples. Most of the diamonds are priced between 500\$ and 999\$ while more expensive diamonds are far less.

In general, we can conclude that this is **a dataset of mid-range diamonds**.

In [131]:
primary_color = "#53E185"
plot = make_histogram(cleaned_dataset, "price", primary_color) 
plot.update_traces(marker_line_width=0, opacity=0.6)
plot["layout"]["xaxis"]["title"] = "price ($)"
plot.update_layout(template="simple_white", bargap=0.15, title="Price Distribution")


### Feature Distribution
**Assumption**: physical measurements unit is millimeters.


As expected from price distribution, also feature distribution is not uniform.
From the plots below we clearly see that most of the diamonds in the dataset are **small nicely cut diamonds**. 

Carat distribution is skewed towards **lower carats** while cut is skewed towards *Ideal* grade. Lower clarity grades are prevalent while color is mostly uniform.

All things considered, we expect **low carat and clarity to play an important role in driving down the price** while cut should counterbalance them by driving price higher but not as much.

In [133]:
from plotly.subplots import make_subplots
import plotly.express as px
from itertools import cycle

primary_color="#53e185"
palette = [primary_color, "#62a6ff", "#ff8f00"]

rows = 3
columns = 3
fig = make_subplots(rows=3, 
                    cols=3, 
                    subplot_titles=[feature.capitalize() for feature in features],
                    vertical_spacing = 0.30,
                    horizontal_spacing = 0.10)

row = 1
column = 1
for feature, color in zip(features, cycle(palette)):
  if feature in categorical_features:
    plot = make_barchart(cleaned_dataset, feature, color)
  else:
    plot = make_histogram(cleaned_dataset, feature, color) 
  fig.add_trace(plot["data"][0], row=row, col=column)
  
  column = (column + 1) % 4
  if column == 0:
    column = 1
    row = row + 1

x_labels=["carats", "grade", "grade", "grade", "millimeters", "millimeters", "millimeters", "millimeters", "millimeters" ]
for plot, x_label in zip(range(1,rows*columns+1), x_labels): 
    fig['layout']['yaxis{}'.format(plot)]['title']='count'
    fig['layout']['xaxis{}'.format(plot)]['title']=x_label
    
# marker_line_width=0 solves a plotly color bug in notebooks
fig.update_traces(marker_line_width=0, opacity=0.6)
fig.update_layout(template="simple_white", bargap=0.15, title="Features Distribution")


## Diamond Price: Main Drivers

### Redundant Features
The first step in assessing the most important features influencing diamonds' prices is computing **correlation**. Correlation measures the **degree of association** between two features.

As a first step we computed the correlation between all features and price. We clearly see that the **most correlated feature is carat**. The more a diamond weights the higher its price, all other features play a lesser role.

From a superficial analysis x,y and z may seem exceptionally important variables as their correlation with price is almost as high as carat.
However, we decided to plot the correlation matrix. Such matrix represents the correlation among all the features (e.g. carat with color, color with table, cut with clarity...) of a diamond. We can clearly see from the lowest-right corner of the matrix that the **correlation among x, y and z is almost 1**. On top of this, all three variables have an almost **perfect correlation with carats** making all three completely redundant with carats.

We decided to **remove x, y and z**



In [112]:
import plotly.express as px
from typing import Literal
import numpy as np

CORRELATION_METHOD = Literal["pearson", "kendall", "spearman"]

def compute_correlation_with_target(dataset: pd.DataFrame, 
                                    method: CORRELATION_METHOD = "pearson") -> pd.Series:
  correlation_dict = {feature:
                      dataset[feature].corr(train.price, method=method)
                      for feature in features}
  return pd.Series(index=features, 
            data=np.abs([dataset[feature].corr(dataset.price) 
            for feature in features])).sort_values(ascending=False)
def plot_correlation_matrix(dataset: pd.DataFrame, method: str ="pearson"):
  correlation_matrix = dataset.corr(method=method).round(2)
  heatmap = px.imshow(correlation_matrix, text_auto=True, color_continuous_scale="viridis")
  heatmap.update_layout(
      font_size=18
  )
  heatmap.update_yaxes(ticksuffix = "  ")
  heatmap.update_xaxes(ticksuffix = "  ")
  return heatmap

In [115]:
print("Correlation between features and price\n")
compute_correlation_with_target(train)

Correlation between features and price



carat      0.922142
y          0.889448
x          0.888563
z          0.881492
color      0.164228
table      0.138904
clarity    0.110467
cut        0.035208
depth      0.023635
dtype: float64

In [117]:
plot_correlation_matrix(train)

In [35]:
samples_train = samples_train.drop(["x", "y", "z"], axis=1)
samples_test = samples_test.drop(["x", "y", "z"], axis=1)

### Most Important Features
After tracking down redundant features we now try to understand which are the most important features when trying to guess diamonds' prices.

We employ one of the simplest machine learning models, **linear regression**. Such model represents price as a weighted combination of the features, such weights are estimated given the data in the training set:

 $\text{price} = w_1*\text{carat} + w_2*\text{color} + ... + w_6*\text{depth}$.

As a first step we try to get a linear regression which performs reasonably well on our data so we try different models. The best performing linear regression in our case is the **ordinary least squares regression**, the simplest of all.

Such model achieves an **accuracy of 89.8%** in guessing the prices of the diamonds in the test set.

Once the training phase is over we have values for $w_1, ..., w_6$ and so we know which are the most important features taken into account by the regression model to decide the price of a given diamond. However, we need also to take into account the standard error, that is how certain the model is about the value attributed to the weight. The weight scaled with the standard error is called **t-statistic**:

$t_{w_i} = \frac{w_i}{\text{standard_error}(w_i)}$
where $w_i$ is any of the weights $w_1, ..., w_6$ and $t_{w_i}$ is the t-statistic of weight $w_i$.

After computing all the t-values we get the following ranking:
- **carat**  ->     189.304804
- **clarity** ->    40.551275
- color     ->  26.598693
- depth     ->  22.561120
- table     ->   7.457210
- cut       ->   4.387625

The exceptional importance of carats is confirmed but we also see that clarity plays the second most important role while table and cut play far lesser roles than all other features.

In [ ]:
from sklearn.linear_model import ElasticNet
import numpy as np
from sklearn.model_selection import RandomizedSearchCV


parameters_grid = {"alpha": np.linspace(0, 1, endpoint=True),
                   "l1_ratio": np.linspace(0, 1)}


regressor = ElasticNet(random_state=0)
grid_search = RandomizedSearchCV(regressor, 
                           parameters_grid, 
                           scoring="r2",
                           cv=10,
                           n_iter=100,
                           verbose=3,
                           random_state=0)
grid_search.fit(samples_train, targets_train)
best_regressor = grid_search.best_estimator_
print("Best regressor R2 score: " 
      + str(best_regressor.score(samples_test,
                                 targets_test)))

In [102]:
print(best_regressor) # with alpha=0 ElasticNet is equivalent to an ordinary least squares (OLS)
print("Best regressor R2 score: " 
      + str(best_regressor.score(samples_test,
                                 targets_test)))

ElasticNet(alpha=0.0, l1_ratio=0.673469387755102, random_state=0)
Best regressor R2 score: 0.8987127314142909


In [103]:
import statsmodels.api as sm
from sklearn.metrics import r2_score
import numpy as np

# statsmodels hosts an OLS implementation with pre-computed t-values

model = sm.OLS(targets_train,samples_train)
results = model.fit()
feature_importance = np.abs(results.tvalues)

In [118]:
print("Feature Importance: \n\n" + str(feature_importance.sort_values(ascending=False)))

Feature Importance: 

carat      189.304804
clarity     40.551275
color       26.598693
depth       22.561120
table        7.457210
cut          4.387625
dtype: float64


## Conclusions
Our analysis concludes with reasonable certainty that in this dataset **carat is the most important feature** for determining a diamond's price. Features **x, y and z are redundant** with carat and highly correlated among themselves thus useless when pricing. **Clarity** is the second most important feature while **cut and table** play a far lesser role than all the others.

Further inspection with a more performant model will be carried out in the next notebook to deepen our understanding of the features driving diamonds' prices.